In [3]:
# mnist_cnn
# MNIST and Convolutional Neural Network
# L1,L2,L3 : conv2d + relu + max_pool 
# L4,L5: FC(Fully Connectedt Layer)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)

In [4]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [5]:
# one-hot 인코딩
nb_classes = 10  # 분류 class의 갯수(0~9)

Y_one_hot = tf.one_hot(y_train,nb_classes)   # (60000, 10)
print(Y_one_hot.shape)                       # (60000, 10) , (2차원)

# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
print(x_train.shape,x_train.dtype)

x_test = tf.cast(x_test,dtype=tf.float32)
print(x_test.shape,x_test.dtype)

(60000, 10)
(60000, 28, 28) <dtype: 'float32'>
(10000, 28, 28) <dtype: 'float32'>


In [6]:
# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_train,[-1,28,28,1])
print(X_img.shape) # (60000, 28, 28, 1)

(60000, 28, 28, 1)


In [7]:
# Layer 1 : conv2d - relu - max_pool
#

# <1> conv2d
# L1 input image shape : (?, 28, 28, 1)
# filter : (3,3,1,32), 필터 32개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (28+2 - 3)/1 + 1 = 28
# (?, 28, 28, 1) --> (?, 28, 28, 32)
W1 = tf.Variable(tf.random.normal([3,3,1,32]),name='weight1')
def L1_conv2d(X):
    return tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L1_relu(X):
    return tf.nn.relu(L1_conv2d(X))  # shape 변화가 없다

# <3> max_pool
# input image : (?, 28, 28, 32)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (28+1 - 2)/2 + 1 = 14
#  (?, 28, 28, 32) -->  (?, 14, 14, 32)
def L1_MaxPool(X):
    return tf.nn.max_pool(L1_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') 

In [8]:
# Layer 2 : conv2d - relu - max_pool
# (?, 14, 14, 32) --> (?,7,7,64)

# <1> conv2d
# L1 input image shape : (?, 14, 14, 32)
# filter : (3,3,32,64), 필터 64개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (14+2 - 3)/1 + 1 = 14
# (?, 14, 14, 32) --> (?, 14,14,64)
W2 = tf.Variable(tf.random.normal([3,3,32,64]),name='weight2') # filter
def L2_conv2d(X):
    return tf.nn.conv2d(L1_MaxPool(X),W2,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L2_relu(X):
    return tf.nn.relu(L2_conv2d(X))  # shape 변화가 없다

# <3> max_pool
# input image : (?, 14,14,64)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (14+1 - 2)/2 + 1 = 7
#  (?, 14,14,64) -->  (?,7,7,64)
def L2_MaxPool(X):
    return tf.nn.max_pool(L2_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') 

# <4> flatten layer : : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L2_flat(X):
    return tf.reshape(L2_MaxPool(X),[-1,7*7*64])

In [9]:
# Layer 3 : conv2d - relu - max_pool
# (?,7,7,64) --> 

# <1> conv2d
# L1 input image shape : (?,7,7,64) 
# filter : (3,3,64,128), 필터 128개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (7+2 - 3)/1 + 1 = 7
# (?,7,7,64)  --> (?, 7, 7 ,128)
W3 = tf.Variable(tf.random.normal([3,3,64,128]),name='weight3') # filter
def L3_conv2d(X):
    return tf.nn.conv2d(L2_MaxPool(X),W3,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L3_relu(X):
    return tf.nn.relu(L3_conv2d(X))  # shape 변화가 없다

# <3> max_pool
# input image : (?, 7,7,128)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (7+1 - 2)/2 + 1 = 4
#  (?,7,7,64) -->  (?,4,4,128)
def L3_MaxPool(X):
    return tf.nn.max_pool(L3_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') 

# <4> flatten layer : : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L3_flat(X):
    return tf.reshape(L3_MaxPool(X),[-1,4*4*128])

In [10]:
# Layer 4 : FC(Fully Connected Layer)
# (?,,4*4*128) * (,4*4*128,512) = (?, 512)
# nb_classes = 10
W4 = tf.Variable(tf.random.normal([4*4*128,512]), name='weight4')
b4 = tf.Variable(tf.random.normal([512]), name='bias4')
def L4_relu(X):
    return tf.nn.relu(tf.matmul(L3_flat(X),W4) + b4)

In [11]:
# Layer 5 : FC(Fully Connected Layer), Final Output Layer
# (?,,512) * (512,10) = (?, 10)
nb_classes = 10
W5 = tf.Variable(tf.random.normal([512,nb_classes]), name='weight5')
b5 = tf.Variable(tf.random.normal([nb_classes]), name='bias5')

In [12]:
# 예측 함수(hypothesis) : H(X) = softmax(W*X + b)
def logits(X):
    return tf.matmul(L4_relu(X),W5) + b5

def hypothesis(X):
    return tf.nn.softmax(logits(X))

In [ ]:
# 방법 2. batch 사이즈로 나누어 학습, 효율적 이며 학습 시간 단축
# 학습 시작

training_epoch = 50
batch_size = 600

# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.keras.optimizers.Adam(lr=0.01)

Y_one_hot = tf.one_hot(y_train,nb_classes)   # (60000, 10)

print('***** Start Learning!!')
for epoch in range(training_epoch): # 50회
    
    avg_cost = 0
    
    # 100 = 60000/600
    total_batch = int(x_train.shape[0]/batch_size)
    for k in range(total_batch):  # 100회
        batch_xs = x_train[0 + k*batch_size:batch_size + k*batch_size]   # 600개의 X 데이터
        batch_ys = Y_one_hot[0 + k*batch_size:batch_size + k*batch_size] # 600개의 Y 데이터
        
        # X값의 shape을 4차원으로 변환
        X_img = tf.reshape(batch_xs,[-1,28,28,1])

        # 비용함수        
        def cost_func_batch():
            cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits(X_img),
                                             labels = batch_ys)
            cost =  tf.reduce_mean(cost_i)
            return cost
        
        # cost를 minimize 한다
        optimizer.minimize(cost_func_batch,var_list=[W1,W2,W3,W4,b4,W5,b5])
        avg_cost += cost_func_batch().numpy()/total_batch
            
    print('Epoch:','%04d'%(epoch + 1),'cost:','{:.9f}'.format(avg_cost))
             
print('***** Learning Finished!!')

***** Start Learning!!
Epoch: 0001 cost: 19067.359869385
Epoch: 0002 cost: 2618.616491089
Epoch: 0003 cost: 1520.005346680


In [ ]:
# 정확도 측정 : accuracy computation

# y_test 값의 one-hot 인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)    # (10000,10)
# print(Y_one_hot.shape)                       # (10000,10)  , (2차원)

# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_test,[-1,28,28,1])

correct_predict = tf.equal(predict(X_img),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))
print("Accuracy:",accuracy.numpy()) # Accuracy: 0.9534

#예측
print('***** Predict')
pred = predict(X_img).numpy()
print(pred,y_test)

In [ ]:
# 정확도 비교

# [1] softmax 사용
# 1 layers              -------> Accuracy  : 0.8871
# 4 layers  sigmoid     -------> Accuracy  : 0.9033
# 4 layers  relu        -------> Accuracy  : 0.9534  

# [2] CNN 사용
# 3 layers              -------> Accuracy  : 0.9721  (epoch=15)
# 3 layers              -------> Accuracy  : 0.9831  (epoch=50)
# 5 layers              -------> Accuracy  :         (epoch=50)